In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

from catboost import CatBoostRegressor
from xgboost import XGBRegressor


import warnings
warnings.filterwarnings("ignore")

In [3]:
df=pd.read_csv("data/StudentsPerformance.csv")

In [4]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [5]:
df.shape

(1000, 8)

In [6]:
df.isna().sum()


gender                         0
race/ethnicity                 0
parental level of education    0
lunch                          0
test preparation course        0
math score                     0
reading score                  0
writing score                  0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [9]:
df.describe()

,math score,reading score,writing score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [10]:
df.nunique()

gender                          2
race/ethnicity                  5
parental level of education     6
lunch                           2
test preparation course         2
math score                     81
reading score                  72
writing score                  77
dtype: int64

In [11]:
categorical_columns=[feature for feature in df.columns if df[feature].dtype=='O']
numerical_columns=[feature for feature in df.columns if df[feature].dtype !='O']

In [12]:
X=df.drop(columns='math score', axis=1)
y=df['math score']

In [13]:
print(X.shape)
print(y.shape)

(1000, 7)
(1000,)


In [14]:
X.head(2)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88


In [15]:
num_features=X.select_dtypes(exclude='object').columns
cat_features=X.select_dtypes(include='object').columns

In [16]:
num_features

Index(['reading score', 'writing score'], dtype='object')

In [17]:
cat_features

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course'],
      dtype='object')

In [16]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

standard_scaler=StandardScaler()
one_hot_encode=OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncode", one_hot_encode, cat_features),
        ("StandardScaler", standard_scaler, num_features)
    ]
)

In [18]:
X=preprocessor.fit_transform(X)

In [19]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [20]:
X.shape

(1000, 19)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [26]:
def evaluate_model(true, predicted):
    mae=mean_absolute_error(true, predicted)
    mse=mean_squared_error(true, predicted)
    rmse=np.sqrt(mse)
    r2=r2_score(true, predicted)
    
    return mae, rmse, r2

In [27]:
models={
    "LinearRegression": LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "Decision Tree": DecisionTreeRegressor(),
    "K Nearest Neighbors": KNeighborsRegressor(),
    "Support Vector": SVR(),
    "Random Forest": RandomForestRegressor(),
    "Cat Boost": CatBoostRegressor(verbose=False),
    "XGBoost": XGBRegressor(),
    "AdaBoost": AdaBoostRegressor(),
}

In [40]:
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2=evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2=evaluate_model(y_test,y_test_pred)

    print(list(models)[i])
    
    print("------------------------")
    print("Model performance for training set: ")
    print("Root Means Square Error: {:.4f}".format(model_train_rmse))
    print("Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("R2 Score: {:.4f}".format(model_train_r2))

    print("------------------------")
    print("Model performance for test set: ")
    print("Root Means Square Error: {:.4f}".format(model_test_rmse))
    print("Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("R2 Score: {:.4f}".format(model_test_r2))

    print("=========================\n")

    model_list.append(list(models.keys())[i])
    r2_list.append(model_test_r2)

LinearRegression
------------------------
Model performance for training set: 
Root Means Square Error: 5.3244
Mean Absolute Error: 4.2671
R2 Score: 0.8743
------------------------
Model performance for test set: 
Root Means Square Error: 5.3960
Mean Absolute Error: 4.2158
R2 Score: 0.8803

Lasso
------------------------
Model performance for training set: 
Root Means Square Error: 6.5938
Mean Absolute Error: 5.2063
R2 Score: 0.8071
------------------------
Model performance for test set: 
Root Means Square Error: 6.5197
Mean Absolute Error: 5.1579
R2 Score: 0.8253

Ridge
------------------------
Model performance for training set: 
Root Means Square Error: 5.3233
Mean Absolute Error: 4.2650
R2 Score: 0.8743
------------------------
Model performance for test set: 
Root Means Square Error: 5.3904
Mean Absolute Error: 4.2111
R2 Score: 0.8806

Decision Tree
------------------------
Model performance for training set: 
Root Means Square Error: 0.2795
Mean Absolute Error: 0.0187
R2 Score: 

In [44]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model', 'R2 Score']).sort_values(by='R2 Score', ascending=False)

,Model,R2 Score
2,Ridge,0.880593
0,LinearRegression,0.880345
9,AdaBoost,0.852231
7,Cat Boost,0.851632
6,Random Forest,0.851630
8,XGBoost,0.827797
1,Lasso,0.825320
4,K Nearest Neighbors,0.783813
3,Decision Tree,0.734362
5,Support Vector,0.728600
